# Activos sin parámetros y/o valores de retorno


Cuando la operación (```Op```) asociada a un activo *B* incluye un parámetro con un nombre que coincide con el nombre de otro activo *A*, Dagster establece una relación que muestra que *B* depende de *A* ```(A -> B)```. Antes de ejecutar la operación del activo *B*, el contenido del activo *A* se carga en memoria desde su última materialización y se pasa como parámetro a dicha operación.

Sin embargo, hay escenarios en los que **NO** es necesario o deseable cargar el contenido de un activo en memoria. Por ejemplo, cuando el activo se "calcula" copiando un archivo, creando una tabla o ejecutando una tarea en una interfaz de línea de comandos ([cmd](https://learn.microsoft.com/en-us/windows-server/administration/windows-commands/cmd), [bash](https://opensource.com/resources/what-bash), [powershell](https://learn.microsoft.com/en-us/powershell/scripting/overview) o similar)

Para esas situaciones, se deben definir activos cuyas operaciones no tienen valores de retorno o argumentos; es posible combinar estos activos con activos que retornan valores o incluyen parámetros. 

Toda función en Python retorna un valor; cuando no se retorna un valor explícitamente, la función retorna ```None```. Para evitar que Dagster materialize ```None``` como el contenido de un activo que NO retorna un valor hay que [anotar la función](https://ellibrodepython.com/function-annotations) de la siguiente manera:

```
@asset
def no_return() -> None:
    print("No explicit return")
```

Finalmente, para establecer una relación de dependencia con un activo que NO retorna un valor, es necesario emplear el parámetro ```non_argument_deps``` del decorador ```@asset```.

In [3]:
#Toda función retorna un valor
def no_return():
    print("No explicit return")
    
print(no_return())

No explicit return
None


## Por defecto, los activos de materializan en formato pickle

In [1]:
import pickle

with open(r"C:\dev\workspace-python\dagster-data-app\tmpyz1ktips\storage\nabisco_cereal_ratings", "rb") as data_asset_pickle_file:
    data_asset = pickle.load(data_asset_pickle_file)

data_asset

{'100% Bran': '68.402973',
 'Cream of Wheat (Quick)': '64.533816',
 'Shredded Wheat': '68.235885',
 "Shredded Wheat 'n'Bran": '74.472949',
 'Shredded Wheat spoon size': '72.801787',
 'Strawberry Fruit Wheats': '59.363993'}

## Ejercicio

Implementar los siguientes requerimientos de la primera prueba de evaluación continua como una aplicación de datos en Dagster.

Para verificar si un paquete Python está instalado en el entorno ```dagster-data-app``` active entorno y ejecute: ```pip list | findStr beautifulsoup4```

**Extraer la masa, densidad, gravedad y duración del día de la siguiente fuente**

In [8]:
import requests

response = requests.get("http://localhost:8000/Planetary_Fact_Sheet.html")
response

<Response [200]>

In [19]:
import pickle
with open(r"C:\Tareas\2022\SisInfo3\dagster\dagster-data-app\tmp8uj0a7n6\storage\namesRow", "rb") as data_asset_pickle_file:
    data_asset_Names = pickle.load(data_asset_pickle_file)
    
data_asset_Names

['MERCURY',
 'VENUS',
 'EARTH',
 'MOON',
 'MARS',
 'JUPITER',
 'SATURN',
 'URANUS',
 'NEPTUNE',
 'PLUTO']

In [11]:
#density
with open(r"C:\Tareas\2022\SisInfo3\dagster\dagster-data-app\tmp8uj0a7n6\storage\densityRow", "rb") as data_asset_pickle_file:
    data_asset_Density = pickle.load(data_asset_pickle_file)

data_asset_Density

[5429.0, 5243.0, 5514.0, 3340.0, 3934.0, 1326.0, 687.0, 1270.0, 1638.0, 1850.0]

In [12]:
#gravity
with open(r"C:\Tareas\2022\SisInfo3\dagster\dagster-data-app\tmp8uj0a7n6\storage\gravityRow", "rb") as data_asset_pickle_file:
    data_asset_gravity = pickle.load(data_asset_pickle_file)

data_asset_gravity

[3.7, 8.9, 9.8, 1.6, 3.7, 23.1, 9.0, 8.7, 11.0, 0.7]

In [13]:
#length 
with open(r"C:\Tareas\2022\SisInfo3\dagster\dagster-data-app\tmp8uj0a7n6\storage\lengthOfDayRow", "rb") as data_asset_pickle_file:
    data_asset_length = pickle.load(data_asset_pickle_file)

data_asset_length

[4222.6, 2802.0, 24.0, 708.7, 24.7, 9.9, 10.7, 17.2, 16.1, 153.3]

In [14]:
#mass
with open(r"C:\Tareas\2022\SisInfo3\dagster\dagster-data-app\tmp8uj0a7n6\storage\massRow", "rb") as data_asset_pickle_file:
    data_asset_mass = pickle.load(data_asset_pickle_file)

data_asset_mass

[0.33, 4.87, 5.97, 0.073, 0.642, 1898.0, 568.0, 86.8, 102.0, 0.013]

In [15]:
#dictionary of all info
def getAllExtractedInfo(namesRow, massRow, densityRow, gravityRow, lengthOfDayRow):
    list_of_lists = [namesRow, massRow, densityRow, gravityRow, lengthOfDayRow]
    planet_dictionary = {z[0]: list(z[1:]) for z in zip(*list_of_lists)} 
    return planet_dictionary

In [16]:
print(getAllExtractedInfo(data_asset_Names,data_asset_mass, data_asset_Density, data_asset_gravity, data_asset_length))

{'MERCURY': [0.33, 5429.0, 3.7, 4222.6], 'VENUS': [4.87, 5243.0, 8.9, 2802.0], 'EARTH': [5.97, 5514.0, 9.8, 24.0], 'MOON': [0.073, 3340.0, 1.6, 708.7], 'MARS': [0.642, 3934.0, 3.7, 24.7], 'JUPITER': [1898.0, 1326.0, 23.1, 9.9], 'SATURN': [568.0, 687.0, 9.0, 10.7], 'URANUS': [86.8, 1270.0, 8.7, 17.2], 'NEPTUNE': [102.0, 1638.0, 11.0, 16.1], 'PLUTO': [0.013, 1850.0, 0.7, 153.3]}


In [17]:
#mass
with open(r"C:\Tareas\2022\SisInfo3\dagster\dagster-data-app\tmp8uj0a7n6\storage\getAllExtractedInfo", "rb") as data_asset_pickle_file:
    data_asset_all = pickle.load(data_asset_pickle_file)

data_asset_all

{'MERCURY': [0.33, 5429.0, 3.7, 4222.6],
 'VENUS': [4.87, 5243.0, 8.9, 2802.0],
 'EARTH': [5.97, 5514.0, 9.8, 24.0],
 'MOON': [0.073, 3340.0, 1.6, 708.7],
 'MARS': [0.642, 3934.0, 3.7, 24.7],
 'JUPITER': [1898.0, 1326.0, 23.1, 9.9],
 'SATURN': [568.0, 687.0, 9.0, 10.7],
 'URANUS': [86.8, 1270.0, 8.7, 17.2],
 'NEPTUNE': [102.0, 1638.0, 11.0, 16.1],
 'PLUTO': [0.013, 1850.0, 0.7, 153.3]}

**Obtener los datos de los tres planetas con mayor duración del día que La Tierra ordenados de manera descendente (por duración del día)**

In [30]:
with open(r"C:\Tareas\2022\SisInfo3\dagster\dagster-data-app\tmp8uj0a7n6\storage\planets_with_most_day_length", "rb") as data_asset_pickle_file:
    data_asset_planets = pickle.load(data_asset_pickle_file)

data_asset_planets

[('MERCURY', [0.33, 5429.0, 3.7, 4222.6]),
 ('VENUS', [4.87, 5243.0, 8.9, 2802.0]),
 ('MOON', [0.073, 3340.0, 1.6, 708.7])]

**Obtener los datos de los tres planetas con menor masa que La Tierra ordenados de manera ascendente por su masa**

In [33]:
with open(r"C:\Tareas\2022\SisInfo3\dagster\dagster-data-app\tmp8uj0a7n6\storage\planets_with_less_mass_than_earth", "rb") as data_asset_pickle_file:
     data_asset_planets = pickle.load(data_asset_pickle_file)

data_asset_planets

[('PLUTO', [0.013, 1850.0, 0.7, 153.3]),
 ('MOON', [0.073, 3340.0, 1.6, 708.7]),
 ('MERCURY', [0.33, 5429.0, 3.7, 4222.6]),
 ('MARS', [0.642, 3934.0, 3.7, 24.7]),
 ('VENUS', [4.87, 5243.0, 8.9, 2802.0])]

# Referencias

- [PEP 3107 – Function Annotations](https://peps.python.org/pep-3107/#fundamentals-of-function-annotations)
